<a href="https://colab.research.google.com/github/dferrero17/RI-Notebook/blob/main/trained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch transformers ranx accelerate
!git clone https://github.com/ua-deti-information-retrieval/Neural-IR-hands-on.git



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.0/318.0 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.3/228.3 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/

In [3]:
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/danielmartinsferreira_ua_pt/EasiDSSJrhxMjiKRT1y52NMBeACFQCCpnDG_CyiKsFwiNQ?e=of9SLk&download=1" -O dev_set_nq_bm25.jsonl
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/danielmartinsferreira_ua_pt/EdWhbTcpyDBJsS-4EH-4DegBYh3jkVIM5ooQENP6vFRZJw?e=wRkIfl&download=1" -O dev_set_nq_gs.jsonl
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/danielmartinsferreira_ua_pt/Ea6gpLL0DJpOhWQjImzL-EkBUR8jBGD67I3vGG4bBH8SYg?e=fPPHUF&download=1" -O train_set_nq_neg.jsonl
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/danielmartinsferreira_ua_pt/Efdw-S2tkVVKrNxpsf80vpgBa93EMfER1xM5P6pcSqjZRw?e=Y3J25w&download=1" -O train_set_nq_pos.jsonl
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/danielmartinsferreira_ua_pt/Eb2ZRZOFAb1BuHeYI6MwtZIBljvlwl_TLN9iWbqhsEHi5w?e=RIc6mG&download=1" -O collection_with_texts.jsonl

--2023-12-21 13:27:46--  https://uapt33090-my.sharepoint.com/:u:/g/personal/danielmartinsferreira_ua_pt/EasiDSSJrhxMjiKRT1y52NMBeACFQCCpnDG_CyiKsFwiNQ?e=of9SLk&download=1
Resolving uapt33090-my.sharepoint.com (uapt33090-my.sharepoint.com)... 13.107.136.10, 13.107.138.10, 2620:1ec:8f8::10, ...
Connecting to uapt33090-my.sharepoint.com (uapt33090-my.sharepoint.com)|13.107.136.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/danielmartinsferreira_ua_pt/Documents/output/dev_set_nq_bm25.jsonl?ga=1 [following]
--2023-12-21 13:27:47--  https://uapt33090-my.sharepoint.com/personal/danielmartinsferreira_ua_pt/Documents/output/dev_set_nq_bm25.jsonl?ga=1
Reusing existing connection to uapt33090-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 965262 (943K) [application/octet-stream]
Saving to: ‘dev_set_nq_bm25.jsonl’

dev_set_nq_bm25.jso 100%[===================>] 942.64K   784KB/s    in 1.2s    

2023-12-21 13:27:49 (784 KB/

# Starting here

In [4]:
!cd Neural-IR-hands-on && git pull

!cp Neural-IR-hands-on/trainer/* .

Already up to date.


In [5]:
from data import get_qrels, InferenceRankingIterator,  BioASQPointwiseIterator, InferenceDataset, create_training_dataset
from sampler import BasicSampler
from transformers import AutoTokenizer

model_checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.model_max_length = 512

# torch.utils.data.Dataset (Iterable)
train_ds = create_training_dataset("train_set_nq_pos.jsonl", # "body"/"question" q_id:[doc_id] 60K
                                   "train_set_nq_neg.jsonl", # q_id:[{"id":doc_id, "score": doc_id}]
                                   "collection_with_texts.jsonl", # doc_id: text (title + " " + abstract)
                                    tokenizer=tokenizer,
                                    iterator_class=BioASQPointwiseIterator[BasicSampler],
                                    #max_questions=500, # debug
                                            )
# torch.utils.data.Dataset (Iterable)
dev_ds = InferenceDataset("dev_set_nq_bm25.jsonl", #q_id:[doc_id ->>] BM25 top-1000 top-100 # 100 question -> 1000 docs
                          train_ds.collection,
                          tokenizer,
                          #max_questions=10, # debug
                          at=100, #max docs
                          gs_path="dev_set_nq_gs.jsonl", # q_id: [doc_id]
                          iterator_class=InferenceRankingIterator)


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
from transformers import AutoModelForSequenceClassification

id2label = {0: "IRRELEVANT", 1: "RELEVANT"}
label2id = {"IRRELEVANT": 0, "RELEVANT": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id
).to("cuda")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
_iter = iter(train_ds)

print(next(_iter))
print(next(_iter))



{'input_ids': [101, 2029, 4295, 2003, 5845, 2007, 21368, 8004, 7630, 19357, 3366, 22989, 1029, 102, 2019, 3350, 1011, 2241, 3319, 1997, 1996, 4022, 6666, 1997, 21368, 8004, 7630, 19357, 3366, 22989, 1999, 1996, 3949, 1997, 5022, 2007, 11721, 22368, 4295, 1012, 11721, 22368, 4295, 2003, 2019, 7900, 1048, 7274, 27642, 5527, 8761, 1010, 7356, 2011, 13366, 20132, 4023, 1997, 1043, 7630, 3597, 17119, 15878, 7352, 8524, 3366, 2877, 2000, 5527, 1997, 1043, 7630, 3597, 17119, 15878, 7352, 5178, 1999, 8153, 26632, 21890, 8449, 1012, 2828, 1045, 4295, 1010, 1996, 2087, 15157, 2433, 1010, 14087, 2430, 6091, 2291, 6624, 2021, 7534, 3952, 2007, 8023, 5445, 1997, 2002, 4502, 13122, 10814, 3630, 4168, 9692, 2100, 1010, 22330, 14399, 19825, 1010, 1998, 5923, 4295, 1012, 26721, 8159, 3560, 9007, 6110, 7242, 2064, 7901, 2122, 24491, 2015, 1012, 1999, 2804, 2000, 1996, 2048, 16285, 2747, 9362, 2005, 2224, 1010, 1996, 14751, 9007, 1010, 21368, 8004, 7630, 19357, 3366, 22989, 1010, 2003, 2012, 1996, 2397, 

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(num_train_epochs=5,
                                  learning_rate=2e-5, # AdamW
                                  weight_decay=0.01,
                                  per_device_train_batch_size=8,
                                  per_device_eval_batch_size=32,
                                  evaluation_strategy= "epoch",
                                  dataloader_pin_memory=True,
                                  output_dir="trained_model",
                                  logging_strategy="steps",
                                  logging_first_step=True,
                                  logging_steps=100,
                                  save_strategy="epoch",
                                  save_total_limit=2,
                                  seed=42)

In [9]:
from ranker_trainer import RankerTrainer
from transformers import DataCollatorWithPadding
from collator import RankingCollator
from metrics import RanxMetrics

import torch
trainer = RankerTrainer(
      model=model,
      args=training_args,
      train_dataset=train_ds,
      eval_dataset=dev_ds, # validation set
      tokenizer=tokenizer,
      data_collator=DataCollatorWithPadding(tokenizer=tokenizer), # apply to train
      eval_data_collator=RankingCollator(tokenizer=tokenizer), # apply to val
      preprocess_logits_for_metrics=lambda logits, labels: torch.nn.functional.softmax(logits, dim=-1)[:,1], # prob for class 1 (relevant class)
      compute_metrics=RanxMetrics(dev_ds.get_qrels()),
  )

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [10]:
trainer.train()

Token indices sequence length is longer than the specified maximum sequence length for this model (974 > 512). Running this sequence through the model will result in indexing errors
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Ndcg@100,Ndcg@10,Recall@10,Map@10
1,0.232300,No log,0.395578,0.380036,0.409751,0.299557
2,0.256500,No log,0.406026,0.390913,0.406851,0.308528
3,0.191300,No log,0.422137,0.408951,0.413847,0.327453
4,0.133600,No log,0.434782,0.424145,0.416911,0.346273
5,0.096500,No log,0.440045,0.430124,0.419626,0.351806


/usr/local/lib/python3.10/dist-packages/ranx/metrics/ndcg.py:72: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  scores[i] = _ndcg(qrels[i], run[i], k, rel_lvl, jarvelin)


TrainOutput(global_step=1595, training_loss=0.21179133537794728, metrics={'train_runtime': 4654.7217, 'train_samples_per_second': 2.735, 'train_steps_per_second': 0.343, 'total_flos': 2929871101406160.0, 'train_loss': 0.21179133537794728, 'epoch': 5.0})

# Use a trained model to run inference!

## TODO reboot the machine

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = "trained_model/checkpoint-1595"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.model_max_length = 512

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint).to("cuda")

In [12]:
from utils import load_collection_lookup
from data import InferenceRankingIterator, InferenceDataset
from collator import RankingCollator
import torch
from tqdm import tqdm
# i am using the dev dataset bc I do not have eval dataset for this example

# load the collection lookup
collection = load_collection_lookup("collection_with_texts.jsonl")

eval_ds = InferenceDataset("dev_set_nq_bm25.jsonl",
                            collection,
                            tokenizer,
                            at=100, #max docs pq q
                            iterator_class=InferenceRankingIterator)

dataloader = torch.utils.data.DataLoader(eval_ds,
                                         batch_size=32,
                                         pin_memory=True,
                                         collate_fn=RankingCollator(tokenizer))

In [13]:
_sample = next(iter(dataloader))

_sample["inputs"].input_ids.shape

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


torch.Size([32, 427])

In [14]:
from collections import defaultdict

run_dict = defaultdict(list) # q_id:[{"doc_id": doc_id, "score": prob}]

for sample in tqdm(dataloader):
  _inputs = sample["inputs"].to("cuda") # send inputs to gpu

  # run inference
  with torch.no_grad():
    logits = model(**_inputs).logits # 0->value, 1->value

    # based on the logits, we normalize to have a probabilistic distribution
    # and we extract the probability of that query-doc pair being relevant!
    prob_relevant_pair = torch.nn.functional.softmax(logits, dim=-1)[:,1] # [0-1]
    # 32,

  for i, q_id in enumerate(sample["id"]):
    run_dict[q_id].append({"doc_id":sample["doc_id"][i],
                           "score":prob_relevant_pair[i].item()})


100%|██████████| 625/625 [11:29<00:00,  1.10s/it]


In [15]:
# lest sort by the neural IR model prob and we have our reranked order!
for q_id in run_dict:
  run_dict[q_id].sort(key=lambda x:-x["score"]) #prob



In [20]:
run_dict

defaultdict(list,
            {'5e48e0e0f8b2df0d49000001': [{'doc_id': '34315200',
               'score': 0.01830712892115116},
              {'doc_id': '34805249', 'score': 0.01257983036339283},
              {'doc_id': '22964750', 'score': 0.010051272809505463},
              {'doc_id': '32725451', 'score': 0.00791869405657053},
              {'doc_id': '27158904', 'score': 0.006341973785310984},
              {'doc_id': '30340822', 'score': 0.005859414115548134},
              {'doc_id': '32924908', 'score': 0.005354179069399834},
              {'doc_id': '30208875', 'score': 0.004691774025559425},
              {'doc_id': '29162322', 'score': 0.003934792242944241},
              {'doc_id': '27334787', 'score': 0.003327065147459507},
              {'doc_id': '24684926', 'score': 0.0032732964027673006},
              {'doc_id': '31436705', 'score': 0.003099966561421752},
              {'doc_id': '22484797', 'score': 0.002969301538541913},
              {'doc_id': '31145129', 'score'